In [148]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [153]:
od = pd.read_excel('..\\data\\raw\\OD 2017\\Tabelas Gerais\\Tab30_OD2017.xlsx', skiprows=7)
od.dropna(inplace=True)
od.rename(columns={'Unnamed: 0' : 'Origem'}, inplace=True)
od = od[od['Origem'] != 'Total']
od = od.drop(columns='Total').astype(int)
od['Total'] = od.drop(columns=['Origem']).sum(axis=1)

In [154]:
od

,Origem,1,2,3,4,5,6,7,8,9,...,509,510,511,512,513,514,515,516,517,Total
0,1,25512,4929,3914,1673,3717,3060,253,90,100,...,0,0,0,256,0,0,0,0,0,160370
1,2,5226,12436,2135,567,1579,3450,2317,0,250,...,0,0,0,543,0,631,405,0,0,137106
2,3,4150,2135,12003,197,351,156,78,156,0,...,0,0,0,0,0,0,0,706,0,62904
3,4,1046,642,197,12182,2931,0,0,409,0,...,0,0,136,0,0,0,0,0,0,78115
4,5,2628,1738,468,3585,35187,5422,210,208,329,...,0,0,300,0,63,0,1174,301,0,186080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,513,0,0,0,0,63,0,0,0,0,...,0,0,44,313,361,91,652,0,0,2953
513,514,0,631,0,0,0,1524,0,0,0,...,0,0,0,736,111,84091,26308,1384,58,156858
514,515,0,405,0,0,1174,498,0,0,0,...,0,0,294,0,633,25963,75177,1108,0,154921
515,516,0,0,706,0,301,301,926,0,0,...,0,0,541,0,0,1384,1667,128438,2313,213561


In [171]:
dg = pd.read_excel('..\\data\\raw\\OD 2017\\Tabelas Gerais\\Dados Gerais OD2017.xlsx', skiprows=6)
dg.columns = dg.columns.str.strip()
dg = dg.dropna()

In [124]:
scaler = MinMaxScaler()

In [194]:
dg_bairros = dg[dg['Nome'].isin(['Pinheiros', 'Moema', 'Butantã'])].copy()
dg_bairros['Zona'] = dg_bairros['Zona'].astype(int)

In [191]:
dg_bairros.columns

Index(['Zona', 'Nome', 'Domicílios', 'Famílias', 'População', 'Matrículas',
       'Empregos', 'Automóveis', 'Produzidas', 'Atraidas', 'Área(ha)'],
      dtype='object')

In [189]:
od_bairros = od[od['Origem'].isin(dg_bairros['Zona'])]

In [199]:
df = pd.merge(left=od_bairros, right=dg_bairros, left_on=['Origem'], right_on=['Zona'], how='inner')
df = df.drop(columns=['Domicílios', 'Famílias', 'Matrículas', 'Empregos', 
                      'Automóveis', 'Produzidas', 'Atraidas', 'Área(ha)'])

In [212]:
dg_bairros

,Zona,Nome,Domicílios,Famílias,População,Matrículas,Empregos,Automóveis,Produzidas,Atraidas,Área(ha)
64,64,Moema,14875.0,14875.000000,33385.987328,3989.673588,41271.791858,15218.5,104605.778721,106168.388724,175.13
82,82,Pinheiros,4937.0,4937.000000,11261.998490,12715.096486,75073.819918,3252.4,163842.163465,164357.023291,202.58
340,340,Butantã,2847.0,3065.839024,7737.997408,1835.700677,13603.161639,3095.91,39505.940371,39510.392852,207.11


In [210]:
od_bairros

,Origem,1,2,3,4,5,6,7,8,9,...,509,510,511,512,513,514,515,516,517,Total
63,64,94,273,189,193,120,240,120,167,0,...,0,0,0,0,0,0,0,648,0,104535
81,82,70,259,473,30,259,256,17,0,26,...,59,0,70,262,63,631,0,1018,82,163712
339,340,518,168,127,0,424,269,0,0,73,...,0,0,0,0,0,0,0,301,0,39442


In [209]:
od[64].sum()

106095

In [211]:
fig = go.Figure()


fig.add_trace(go.Bar(
    x=df['Nome'],
    y=df.drop(columns=['Origem', 'Nome', 'População', 'Total']).sum(axis=1),
    name='Viagens Produzidas',
    marker_color='lightsalmon'
))

fig.add_trace(go.Bar(
    x=df['Nome'],
    y=df['População'],
    name='População',
    marker_color='indianred'
))

In [125]:
fig = go.Figure()


fig.add_trace(go.Bar(
    x=dg['Nome'],
    y=scaler.fit_transform(od.drop(columns='Origem').sum(axis=1).values.reshape(-1, 1))[:,0],
    name='Viagens Produzidas',
    marker_color='lightsalmon'
))


fig.add_trace(go.Bar(
    x=dg['Nome'],
    y=scaler.fit_transform(dg['População'].values.reshape(-1, 1))[:,0],
    name='População',
    marker_color='indianred'
))

In [143]:
fig = go.Figure()


fig.add_trace(go.Bar(
    x=dg['Nome'],
    y=od.drop(columns=['Origem', 'Total']).sum(axis=1) / dg['População'],
    name='Viagens Produzidas',
    marker_color='lightsalmon'
))

In [139]:
tmp = od.drop(columns=['Origem', 'Total']).copy()
np.fill_diagonal(tmp.values, 0)

In [141]:
tmp['Origem'] = np.array(tmp.index) + 1

In [146]:
dg['População']

1        3262.000949
2        5695.000762
3       17130.000980
4       24088.998880
5       14955.999880
           ...      
513      3587.000811
514    104625.996713
515    120821.008206
516    131607.978849
517     18107.003304
Name: População, Length: 517, dtype: float64

In [147]:
od.iloc[0]

Origem         1
1          25512
2           4929
3           3914
4           1673
           ...  
514            0
515            0
516            0
517            0
Total     160370
Name: 0, Length: 519, dtype: int64

In [144]:
fig = go.Figure()


fig.add_trace(go.Bar(
    x=dg['Nome'],
    y=od.drop(columns='Origem').sum(axis=1) / dg['População'],
    name='Viagens Produzidas',
    marker_color='lightsalmon'
))